For kernel errors:
 
    pip install --upgrade pywin32==224

https://stackoverflow.com/questions/58612306/how-to-fix-importerror-dll-load-failed-while-importing-win32api

In [1]:
%load_ext autoreload
%autoreload 2
# import importlib
# importlib.reload(assignment5.markov)

In [2]:
import sys
sys.path.append('..')
from assignment5.utils import get_seqs_from_file, get_reverse_complement, get_cds_from_file
from assignment5.orf import ORFFinder, ORFAnalyzer
from assignment5.markov import GeneModel
import pandas as pd
import numpy as np

## Fetch Sequence

In [3]:
genome = get_seqs_from_file('../data/GCF_000091665.1_ASM9166v1_genomic.fna')[0]
finder = ORFFinder(genome)
orfs = [finder.get_all_orfs(rf ) for rf in (1,2,3)]
orfs = pd.concat(orfs,ignore_index=True)
#orfs = raw_orfs[raw_orfs.length>=5].copy()
orfs.head()

,start,end,length,frame
0,1,36,36,1
1,40,51,12,1
2,55,72,18,1
3,76,81,6,1
4,85,87,3,1


## Fetch Label

In [16]:
cds = np.array(get_cds_from_file('../data/GCF_000091665.1_ASM9166v1_genomic.gff'))
true_orf_end = set(end-3 if  genome[end-3:end] in finder.stop_codons else end for end in cds[:,1])
len(true_orf_end)

848

## Create Markov model

In [71]:
analyzer = ORFAnalyzer(orfs)
long_orfs = analyzer.get_long_ofs()
positive_seqs = finder.get_sequences(long_orfs)
bg_seqs = [get_reverse_complement(seq) for seq in positive_seqs]

model = GeneModel(k = 5, pseudo_count=1)
model.build(positive_seqs, bg_seqs)


In [72]:
model.print_sample_counts()

Foreground T|AAGxy counts
     A   C    G    T
A  307  51  223  394
C  104  15   12  119
G  211  42   39  218
T  148  19   68  198
Background T|AAGxy counts
     A   C   G    T
A   90  26  48   95
C   87  22  15  119
G   41  20  26   59
T  139  39  64  175


## Merge Everything

In [73]:

truth = [True if end in true_orf_end else False for end in orfs.end]
orfs['isCDS'] = truth

orf_sequences = finder.get_sequences(orfs)
scores = [model.get_loglikelihood_ratio(seq) if len(seq)>=5 else np.nan for seq in orf_sequences ]
orfs['scores'] = scores

orfs.head()

,start,end,length,frame,isCDS,scores
0,1,36,36,1,False,1.181405
1,40,51,12,1,False,2.758633
2,55,72,18,1,False,-2.459251
3,76,81,6,1,False,0.568646
4,85,87,3,1,False,NaN


# Results

In [74]:
for rf in (1,2,3):
    rf_orf = orfs[orfs.frame == rf]
    print('\nReading Frame : ', rf)
    print( 'Number of ORFs = ', len(rf_orf))
    print('Summary of the first and last : ')
    display(rf_orf.sort_values('start').iloc[[0,-1]])


Reading Frame :  1
Number of ORFs =  35200
Summary of the first and last : 


,start,end,length,frame,isCDS,scores
0,1,36,36,1,False,1.181405
35199,1664968,1664970,3,1,False,NaN



Reading Frame :  2
Number of ORFs =  35933
Summary of the first and last : 


,start,end,length,frame,isCDS,scores
35200,2,94,93,2,False,2.557823
71132,1664921,1664968,48,2,False,-1.381238



Reading Frame :  3
Number of ORFs =  35686
Summary of the first and last : 


,start,end,length,frame,isCDS,scores
71133,3,5,3,3,False,NaN
106818,1664964,1664969,6,3,False,1.869675


In [75]:
print('The total number of short ORFs = ', len(orfs[orfs.length<50]) )
print('The total number of long ORFs = ', len(orfs[orfs.length>1400]) )

The total number of short ORFs =  81738
The total number of long ORFs =  118


In [76]:
print("The total number of simple plus strand CDSs found in GenBank = ", len(true_orf_end))

The total number of simple plus strand CDSs found in GenBank =  848


In [77]:
print('P(T | AAGxy), Q(T | AAGxy) for the 16 possible combinations of x,y in A,C,G,T :')
model.print_sample_counts()

P(T | AAGxy), Q(T | AAGxy) for the 16 possible combinations of x,y in A,C,G,T :
Foreground T|AAGxy counts
     A   C    G    T
A  307  51  223  394
C  104  15   12  119
G  211  42   39  218
T  148  19   68  198
Background T|AAGxy counts
     A   C   G    T
A   90  26  48   95
C   87  22  15  119
G   41  20  26   59
T  139  39  64  175


In [78]:
print('summary data for the first 5 short ORFs')
display(orfs[orfs.length<50].sort_values('start').head(5))

print('summary data for the the first 5 long ORFs')
display(orfs[orfs.length>1400].sort_values('start').head(5))

summary data for the first 5 short ORFs


,start,end,length,frame,isCDS,scores
0,1,36,36,1,False,1.181405
71133,3,5,3,3,False,NaN
71134,9,20,12,3,False,-0.463650
71135,24,32,9,3,False,0.890789
1,40,51,12,1,False,2.758633


summary data for the the first 5 long ORFs


,start,end,length,frame,isCDS,scores
71526,17619,19229,1611,3,True,166.509569
36031,33626,35245,1620,2,True,208.460996
36169,42725,45109,2385,2,True,258.774328
72661,74592,76010,1419,3,True,138.186040
36888,76820,78481,1662,2,True,202.815509


### CDSs without stop codon at the end

In [79]:
print(f'ORFS which are CDS = {orfs.isCDS.sum()}, Total CDSs = {len(true_orf_end)}')
tend = true_orf_end.copy()
#stop_codons = {'TAA', 'TAG', 'TGA'}
for x in tend-set(orfs.end):
    print(x, genome[x-3:x], genome[x-3:x+3])

ORFS which are CDS = 842, Total CDSs = 848
755683 TCG TCGTTA
742666 CCA CCACTG
754669 AAA AAATCA
1563085 TTT TTTTGG
753619 GAT GATAAA
15774 GGT GGTTCG
